In [1]:
import pandas as pd
import pprint as pprint
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import config
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [2]:
# start date
start_date = datetime.today() - relativedelta(days=1)
end_date = start_date + relativedelta(days=6)

# difference between each date. D means one day
D = 'D'

date_list = pd.date_range(start_date, end_date, freq=D)
keys = []

for date in date_list:
    date_formatted = str(date.date()).replace('-','_')  + '_' + str(date.day_of_week)
    keys.append(date_formatted + '_start')
    keys.append(date_formatted + '_end')

# Add a key for the full name in front
keys.insert(0, 'full_name')

# Create dictionary for coworkers' schedules for one week
coworker_schedule_dict = {key: [] for key in keys}
coworker_schedule_dict

{'full_name': [],
 '2025_02_02_6_start': [],
 '2025_02_02_6_end': [],
 '2025_02_03_0_start': [],
 '2025_02_03_0_end': [],
 '2025_02_04_1_start': [],
 '2025_02_04_1_end': [],
 '2025_02_05_2_start': [],
 '2025_02_05_2_end': [],
 '2025_02_06_3_start': [],
 '2025_02_06_3_end': [],
 '2025_02_07_4_start': [],
 '2025_02_07_4_end': [],
 '2025_02_08_5_start': [],
 '2025_02_08_5_end': []}

In [3]:
if __name__ == "__main__":
    try:
        # instantiate driver
        driver = webdriver.Chrome()

        # visit the target URL
        driver.get(config.target_url)
        time.sleep(0.5)
        
        # fill in the username field
        username = driver.find_element(By.ID, "username")
        username.send_keys(config.username_key)
        time.sleep(0.6)

        # fill in the password field
        password = driver.find_element(By.ID, "password")
        password.send_keys(config.password_key)

        # submit the login form
        driver.find_element(By.ID, "btnLogin").click()
        time.sleep(1) # keep here, need it to load element before clicking

        # find and click on the menu button
        driver.find_element(By.CLASS_NAME, "navbar-toggle").click()
        time.sleep(1)

        # click on 'My Schedule' from the tab
        navbar_schedule = driver.find_elements(By.CSS_SELECTOR, "span[ng-bind='::item.name']")[4]
        driver.find_element(By.LINK_TEXT, navbar_schedule.text).click()
        time.sleep(0.5)

        # click on 'Co-Workers' on the top of the page
        driver.find_element(By.LINK_TEXT, "Co-Workers").click()
    
        print("Successfully made it to the coworker page!!")
        time.sleep(4)

        # set the minHeight of the coworker table to 2000px
        attribute_value = "coworker-header"
        driver.execute_script(f"document.querySelector('div[calc-size-id={attribute_value}]').style.minHeight = '2000px';")
        
        # loop through the entire table by adding data to the dictionary and scrolling down the page
        for i in range(4):
            print("("*50)
            print(f'Loop {i+1}')
            print(")"*50)
            time.sleep(2)

            # set table element for scrolling
            element_table = driver.find_element(By.CSS_SELECTOR, f"div[calc-size-id={attribute_value}]")

            # find all the rows on the page
            table_rows = element_table.find_elements(By.CSS_SELECTOR, "tr")
            for row in table_rows:
                # check if a row is empty
                if not row.text:
                    print("empty row")
                    continue

                # find the full name of coworker
                full_name = row.find_element(By.CSS_SELECTOR, "td.empCell").text

                # clean up and hide the full name for privacy
                split_full_name = full_name.split(',')
                hidden_last_name = split_full_name[0][0:2]
                hidden_first_name = split_full_name[1].replace(' ', '')[0:2]
                hidden_full_name = f'{hidden_last_name}, {hidden_first_name}'
                
                # add name to the dictionary jf it's not already in it
                if hidden_full_name not in coworker_schedule_dict['full_name']:
                    coworker_schedule_dict['full_name'].append(hidden_full_name)
            
                    print("#"*50)
                    print("FULL NAME:", hidden_full_name)
                    print("#"*50)

                    # find all the shift times
                    shift_cells = row.find_elements(By.CSS_SELECTOR, "td.text-on-date-cell-view")
                    day_of_week = 6
                    relativedelta_days = 0

                    # Three types of cells: LTA, Not Scheduled, or normal shift time
                    for cell in shift_cells:          
                        key_start = f'{str(start_date.date() + relativedelta(days=relativedelta_days)).replace("-","_")}_{day_of_week}_start'
                        key_end = f'{str(start_date.date() + relativedelta(days=relativedelta_days)).replace("-","_")}_{day_of_week}_end'

                        # Check for any holidays in the schedule
                        try:
                            holiday_check = cell.find_element(By.CLASS_NAME, "mob-employee-holiday").text
                            print(f'Holiday: {holiday_check} !!')
                            coworker_schedule_dict[key_start].append(holiday_check)
                            coworker_schedule_dict[key_end].append(holiday_check)
                            if day_of_week == 6:
                                day_of_week = 0
                            else:
                                day_of_week+=1
                            relativedelta_days+=1
                            
                        except Exception as no_holiday:
                            # check if it contains a LTA logo
                            try:
                                lta_check = cell.find_element(By.TAG_NAME, "svg")
                                if lta_check.get_attribute("role") == "presentation":
                                    print("LTA")
                                    print("-"*50)
                                    # Add "LTA" as entry for shift start and end times to dictionary
                                    coworker_schedule_dict[key_start].append("LTA")
                                    coworker_schedule_dict[key_end].append("LTA")
                                    if day_of_week == 6:
                                        day_of_week = 0
                                    else:
                                        day_of_week+=1
                                    relativedelta_days+=1
                            
                            except Exception as no_lta:
                                # check if coworker is simply not scheduled
                                if cell.text == "Not scheduled":
                                    print("NOT SCHEDULED")
                                    print("-"*50)
                                    # add "Not Scheduled" as entry for shift start and end times to dictionary
                                    coworker_schedule_dict[key_start].append("Not Scheduled")
                                    coworker_schedule_dict[key_end].append("Not Scheduled")
                                    if day_of_week == 6:
                                        day_of_week = 0
                                    else:
                                        day_of_week+=1
                                    relativedelta_days+=1
                                    continue
                                
                                # check if the cell is empty
                                if not cell.text:
                                    print("should scroll before continuing")
                                    break

                                print(cell.text)
                                print("-"*50)
                                shift_start_attribute = "SHIFT_START"
                                shift_end_attribute = "SHIFT_END"

                                # find the shift start and end times
                                shift_start = cell.find_element(By.CSS_SELECTOR, f"span[mob-field-sec={shift_start_attribute}]").text
                                shift_end_list = cell.find_elements(By.CSS_SELECTOR, f"span[mob-field-sec={shift_end_attribute}]")
                                
                                # add the later end time if coworker has two end times (most likely for shifts longer than 8 hours)
                                if len(shift_end_list) > 1:
                                    shift_end = shift_end_list[1].text
                                else:
                                    shift_end = shift_end_list[0].text

                                # Format times to code %I:%M%p
                                if shift_start.endswith('a'):
                                    shift_start = shift_start.replace('a', 'AM')
                                elif shift_start.endswith('p'):
                                    shift_start = shift_start.replace('p', 'PM')
                                if shift_end.endswith('a'):
                                    shift_end = shift_end.replace('a', 'AM')
                                elif shift_end.endswith('p'):
                                    shift_end = shift_end.replace('p', 'PM')
                                
                                # add the shift start and end times to the dictionary
                                coworker_schedule_dict[key_start].append(shift_start)
                                coworker_schedule_dict[key_end].append(shift_end)
                                if day_of_week == 6:
                                        day_of_week = 0
                                else:
                                    day_of_week+=1
                                relativedelta_days+=1

                # check if the coworker's full schedule is already added to the dictionary
                elif not all(len(item) == len(list(coworker_schedule_dict.values())[0]) for item in coworker_schedule_dict.values()):
                    print("The array lengths are uneven!!")
                    coworker_schedule_dict['full_name'].pop()
                else:
                    print("Full name is in dictionary and all lists are same length")

            # scroll down to load additional rows
            driver.execute_script("arguments[0].scrollIntoView(false);", element_table)

        # log out of the session
        driver.find_element(By.CLASS_NAME, "navbar-toggle").click()
        time.sleep(1)

        driver.find_element(By.LINK_TEXT, "Log Out").click()
        time.sleep(1)

        # close the browser
        driver.quit()

    except Exception as e:
        # print the error message for debugging
        print("Error with code, website, or both!!", e)

        # log out of the session
        driver.find_element(By.CLASS_NAME, "navbar-toggle").click()
        time.sleep(1)

        driver.find_element(By.LINK_TEXT, "Log Out").click()
        time.sleep(1)

        # close the browser
        driver.quit()


Successfully made it to the coworker page!!
((((((((((((((((((((((((((((((((((((((((((((((((((
Loop 1
))))))))))))))))))))))))))))))))))))))))))))))))))
##################################################
FULL NAME: AL, AN
##################################################
4:00p-8:00p
--------------------------------------------------
1:00p-5:00p
--------------------------------------------------
NOT SCHEDULED
--------------------------------------------------
8:00a-12:00p
--------------------------------------------------
NOT SCHEDULED
--------------------------------------------------
NOT SCHEDULED
--------------------------------------------------
NOT SCHEDULED
--------------------------------------------------
##################################################
FULL NAME: AR, AN
##################################################
3:00p-7:00p
--------------------------------------------------
NOT SCHEDULED
--------------------------------------------------
NOT SCHEDULED
---------------

In [4]:
# check if all lists' lengths are equal
[len(coworker_schedule_dict[key]) for key in coworker_schedule_dict.keys()]

[49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49]

In [5]:
# DON'T RUN CELL IF TODAY IS FEBRUARY 9 2025 OR LATER
# Create it one time on the week of 02/02/2025
test_workaround_df = pd.DataFrame(coworker_schedule_dict)
test_workaround_df


# test_workaround_df = pd.DataFrame(dict([(key, pd.Series(value)) for key, value in coworker_schedule_dict.items()]))

,full_name,2025_02_02_6_start,2025_02_02_6_end,2025_02_03_0_start,2025_02_03_0_end,2025_02_04_1_start,2025_02_04_1_end,2025_02_05_2_start,2025_02_05_2_end,2025_02_06_3_start,2025_02_06_3_end,2025_02_07_4_start,2025_02_07_4_end,2025_02_08_5_start,2025_02_08_5_end
0,"AL, AN",4:00PM,8:00PM,1:00PM,5:00PM,Not Scheduled,Not Scheduled,8:00AM,12:00PM,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled
1,"AR, AN",3:00PM,7:00PM,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,4:00PM,8:00PM,3:00PM,7:00PM
2,"BA, CH",6:00AM,3:00PM,Not Scheduled,Not Scheduled,9:00AM,6:00PM,12:00PM,9:00PM,6:00AM,3:00PM,Not Scheduled,Not Scheduled,6:00AM,3:00PM
3,"BE, ED",LTA,LTA,LTA,LTA,10:00AM,7:00PM,6:00AM,3:00PM,9:00AM,6:00PM,9:00AM,6:00PM,10:00AM,7:00PM
4,"CA, TA",11:00AM,3:00PM,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled
5,"CA, GI",Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,2:00PM,6:00PM,3:00PM,7:00PM
6,"CA, SI",Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,5:00PM,9:00PM,6:00PM,10:00PM,3:00PM,7:00PM
7,"CA, CR",2:00PM,6:00PM,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled
8,"DI, KE",Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,8:00AM,12:00PM,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled
9,"ES, FI",4:00PM,8:00PM,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled,Not Scheduled


In [6]:
# RUN THIS CELL INSTEAD
# set full name as index and use join() to add schedules weekly every sunday

In [8]:
# clean up start_date before checking for overlaps
clean_start_date = start_date.replace(hour=0, minute=0, second=0, microsecond=0)

In [9]:
# LATEST: FRACTIONAL HOURS WORKING, NEEDS SOME CLEANUP WITH .HOUR, .MINUTE, AND .TIME()
# myStart < coworkerEnd and coworkerStart < myEnd, then has overlapping shift
for i in range(len(test_workaround_df['2025_02_04_1_start'])):
    test_coworker_start = test_workaround_df['2025_02_04_1_start'][i]
    test_coworker_end = test_workaround_df['2025_02_04_1_end'][i]
    if (test_coworker_start or test_coworker_end) == 'Not Scheduled' or (test_coworker_start or test_coworker_end) == 'LTA':
        continue
    else:
        test_coworker_time_start = datetime.strptime(test_coworker_start, '%I:%M%p')
        test_coworker_time_end = datetime.strptime(test_coworker_end, '%I:%M%p')
        # change test_my_start and test_my_end to times from my schedule
        test_my_start = (clean_start_date + relativedelta(days=2,hours=17))
        test_my_end = (clean_start_date + relativedelta(days=2,hours=21))

        if  test_my_start.time() < test_coworker_time_end.time() and test_coworker_time_start.time() < test_my_end.time():
            overlap_start = max(test_coworker_time_start.hour + (test_coworker_time_start.minute)/60, test_my_start.hour + (test_my_start.minute)/60)
            overlap_end = min(test_coworker_time_end.hour + (test_coworker_time_end.minute)/60, test_my_end.hour + (test_my_end.minute)/60)
            overlap = overlap_end - overlap_start
            print("THERE IS OVERLAP", overlap_start, overlap_end, test_workaround_df.loc[i,'full_name'])
            print(f'OF {overlap} HOURS')
        else:
            continue

THERE IS OVERLAP 17.0 18.0 BA, CH
OF 1.0 HOURS
THERE IS OVERLAP 17.0 19.0 BE, ED
OF 2.0 HOURS
THERE IS OVERLAP 17.0 21.0 FU, DE
OF 4.0 HOURS
THERE IS OVERLAP 17.0 20.0 LE, EL
OF 3.0 HOURS
THERE IS OVERLAP 17.0 21.0 RI, LI
OF 4.0 HOURS
THERE IS OVERLAP 17.0 21.0 TU, TI
OF 4.0 HOURS
